In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import argparse
import os.path
import io
import os
import open3d as o3d
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from sklearn.cluster import DBSCAN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_resnet50_test_1' 
PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-17')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [3]:
def NumpyToPCD(xyz):
    """Function to convert numpy array to open3d point cloud 
        Input: xyz - numpy array
        Output: pcd - Point Cloud data"""

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    return pcd


def detect_planes(input_pcd, min_ratio):
    """Function to detect multiple planes using RANSAC algorithm"""
    pcd = np.asarray(input_pcd.points)

    inplanes = []
    dist_threshold=0.005
    ransac_n=20
    num_iter=1000

    N = len(pcd)
    count = 0

    while (count < (1 - min_ratio)*N):
        pcd = NumpyToPCD(pcd)
        plane_model, inliers = pcd.segment_plane(dist_threshold, ransac_n, num_iter)
        count += len(inliers)
        pcd = np.asarray(pcd.points)
        inplanes.append((plane_model, pcd[inliers]))
        pcd = np.delete(pcd, inliers, axis=0)
    
    return inplanes

def feature_calc(result):
    # Initialize required arrays
    planes = []
#     colors = []
    equations = []
    tilt_angle = []

    # Target plane to calculate plane angle
    xz_plane = np.array([0, 1, 0])

    for eq, plane in results:

        # Initiate random rgb values for different planes
#         r = random.random()
#         g = random.random()
#         b = random.random()

#         color = np.zeros((plane.shape[0], plane.shape[1]))
#         color[:, 0] = r
#         color[:, 1] = g
#         color[:, 2] = b

        # Calculating tilt angles of the planes detected
        vec1 = [eq[0], eq[1], eq[2]]
        num = np.dot(vec1,xz_plane)
        denom = np.linalg.norm(vec1)*np.linalg.norm(xz_plane)
        theta = np.arccos(num/denom)

        # Adding the values to the appropriate arrays
        tilt_angle.append(theta)
        planes.append(plane)
#         colors.append(color)
        equations.append(eq)

    # Reshaping the array for clustering algorithm
    tilt_angle = np.asarray(tilt_angle).reshape(-1,1)

    # Using clustering algorithm to identify anomalies
    cluster = DBSCAN(eps=0.03,min_samples=1).fit(tilt_angle)
    labels = cluster.labels_

    # Returning the labels with most counts and identifying its index
    label,counts = np.unique(labels,return_counts = True)
    index = np.where(counts==np.max(counts))[0][0]
    cluster_index = label[index]

    # Initializing required arrays
    eq_result = []
    points = []
#     rgb = []
    count = 0

    # Retrieving the equation of the planes that correspond to labels from clustering algorithm
    for i in labels:
        if (i == cluster_index):
            eq_result.append(equations[count])
            points.append(planes[count])
#             rgb.append(colors[count])
        count += 1

    # Concatenating the arrays for visualization purposes
    points = np.concatenate(points, axis=0)
#     rgb = np.concatenate(rgb, axis=0)

    # Visualization of the Point Cloud data
    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(points)
    # pcd.colors = o3d.utility.Vector3dVector(rgb)
    # o3d.visualization.draw_geometries([pcd])

    # Calculating the Tread and Riser dimensions
    y_mean = np.average(points[:,1])
    z_mean = np.average(points[:,2])
    y25 = np.percentile(points[:,1],25)
    y75 = np.percentile(points[:,1],75)
    z25 = np.percentile(points[:,2],25)
    z75 = np.percentile(points[:,2],75)

    y1 = []
    y2 = []
    y3 = []
    z1 = []
    z2 = []
    z3 = []

    for a,b,c,d in eq_result:
        y1.append((-c*z_mean - d)/b) 
        z1.append((-y_mean*b - d)/c)
        y2.append((-c*z25 - d)/b) 
        z2.append((-y25*b - d)/c)
        y3.append((-c*z75 - d)/b) 
        z3.append((-y75*b - d)/c)
    #     print("y = {}x + {}".format(-b/c,-d/c))


    tread = ((np.max(z1) - np.min(z1)) + (np.max(z2) - np.min(z2)) + (np.max(z3) - np.min(z3))) * (1/3)
    riser = ((np.max(y1) - np.min(y1)) + (np.max(y2) - np.min(y2)) + (np.max(y3) - np.min(y3))) * (1/3)
    
    return tread,riser

# Read Bag File

In [7]:
bagfilename = "case3.bag"

try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, bagfilename)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.rgb8, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Intel Realsense", cv2.WINDOW_AUTOSIZE)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
        frames = pipeline.wait_for_frames()
        align_to = rs.stream.color
        align = rs.align(align_to)
        aligned_frames = align.process(frames)
        
        # Obtain Camera Intrinsic Parameters 
        intrinsics = aligned_frames.profile.as_video_stream_profile().intrinsics
        pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
            intrinsics.width, intrinsics.height, intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy)
        
        # Wait for a coherent pair of frames: depth and color
        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Perform object detection on color image from camera stream
        input_tensor = tf.convert_to_tensor(np.expand_dims(color_image, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = color_image.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=1,
                    min_score_thresh=.5,    # Change this threshold value as needed
                    agnostic_mode=False)    
        
        height, width, channels = color_image.shape
        ymin, xmin, ymax, xmax = detections['detection_boxes'][0]
        bbox = [int(ymin*height), int(xmin*width), int(ymax*height), int(xmax*width)]
        
        if (detections['detection_scores'][0] > 0.85):
            print("[INFO] Stairs Detected")

            # Crop RGB Image
            color_crop_img_np = color_image[bbox[0]:bbox[2], bbox[1]:bbox[3]]
            cv2.imwrite('test_color.png',color_crop_img_np)

            # Crop Depth Image
            depth_crop_img_np = depth_image[bbox[0]:bbox[2], bbox[1]:bbox[3]]
            cv2.imwrite('test_depth.png',depth_crop_img_np)
            
            # Feature Extractopm
            img_color = o3d.io.read_image('test_color.png')
            img_depth = o3d.io.read_image('test_depth.png')

            #Create RGBD image
            rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(img_color, img_depth)


            # Create Point Cloud
            pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, pinhole_camera_intrinsic)


            # Downsample the point cloud
            pcd = pcd.voxel_down_sample(voxel_size=0.04)

            counter = 0

            # Initialize parameters 
            min_ratio = 0.1

            # Using RANSAC to detect planes 
            results = detect_planes(pcd, min_ratio)

            # Dimension calculation
            tread, riser = feature_calc(results)


            print("[INFO] Tread dimensions = %.2f m"%tread)
            print("[INFO] Riser dimensions = %.2f m"%riser)
            

        cv2.namedWindow('RealSense',cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense',image_np_with_detections)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break  

finally:
    pipeline.stop()
    print("[INFO] Streaming Stopped.")
    cv2.destroyAllWindows()

[INFO] Streaming Stopped.


In [17]:
img_color = o3d.io.read_image('test_color.png')
img_depth = o3d.io.read_image('test_depth.png')

#Create RGBD image
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(img_color, img_depth)

In [ ]:
bagfilename = "case3.bag"

try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, bagfilename)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.rgb8, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Intel Realsense", cv2.WINDOW_AUTOSIZE)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
         # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_img = np.asanyarray(color_frame.get_data())
        color_image = cv2.cvtColor(color_img, cv2.COLOR_RGB2BGR)

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        depth_colormap_dim = depth_colormap.shape
            
        # Perform object detection on color image from camera stream
        input_tensor = tf.convert_to_tensor(np.expand_dims(color_image, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = color_image.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.5,    # Change this threshold value as needed
                    agnostic_mode=False)    
            
        color_colormap_dim = image_np_with_detections.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(image_np_with_detections, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        cv2.imwrite('case3_rgb.png',color_image)
        cv2.imwrite('case3_colored_depth.png',depth_colormap)
        cv2.imwrite('case3_depth.png',depth_image)
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

finally:
    pass